In [1]:
import pandas as pd
import datetime

In [2]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

#date_str = '2018-05-31'

print("Date: " + date_str)

Date: 2018-06-25


In [3]:
# Read the Calendar extract to get additional details
cal_1 = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_calendar/stayz_calendar_' + date_str + '.json')



cal_2 = cal_1[cal_1['property_id'].notnull()]

cal_2.set_index('property_id',inplace=True)


cal = cal_2

cal.head()

,calendar,ext_at,features,photos,review_count,review_value
property_id,,,,,,
9219862,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-25 19:51:05,Books\nChild's high chair\nChildren welcome\nC...,24,8,4.9
9155220,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-25 19:51:10,Air conditioning\nBooks\nChild's high chair\nC...,20,37,-1.0
9188329,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-25 19:51:13,Books\nClothes dryer\nDeck/Patio\nDining\nDish...,11,3,5.0
9174814,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-25 19:51:19,Air conditioning\nChild's high chair\nChildren...,20,8,4.8
9062114,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-25 19:51:26,Air conditioning\nBalcony\nBooks\nChildren wel...,19,15,-1.0


In [4]:
position_dict = {}
position_index = 0


data_dict = {}


# Step one - Create the dictionary of all possible feature items
for index, row in cal.iterrows():
    
    a = row['features']
    
    #print("Features: " + a)
    #save_features = [0] * len(position_dict)
    
    b = a.split('\n')
    
    for b1 in b:
        # Check if it exists in the dictionary
        
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                #save_features[b1_pos] = 1

            else:
                position_dict[b1] = position_index

                b1_pos = position_index

                position_index += 1
            
print(position_dict)



{'Books': 0, "Child's high chair": 1, 'Children welcome': 2, 'Clothes dryer': 3, 'Dining': 4, 'Dishes and utensils': 5, 'DVD player': 6, 'Games': 7, 'Hair dryer': 8, 'Heating': 9, 'Iron and board': 10, 'Kitchen': 11, 'Lawn/Garden': 12, 'Linens provided': 13, 'Living room': 14, 'Microwave': 15, 'Non-Smoking': 16, 'Outdoor grill': 17, 'Oven': 18, 'Parking': 19, 'Refrigerator': 20, 'Satellite/Cable': 21, 'Stereo': 22, 'Stove': 23, 'Television': 24, 'Toaster': 25, 'Towels provided': 26, 'Washing machine': 27, 'Wheelchair accessible': 28, 'Wood stove': 29, 'Air conditioning': 30, 'Coffee maker': 31, 'Deck/Patio': 32, 'Dishwasher': 33, 'Fireplace': 34, 'Games room': 35, 'Internet access': 36, 'Video library': 37, 'Garage': 38, 'Balcony': 39, 'Events allowed': 40, 'Telephone': 41, 'Long-term renters welcome': 42, 'Pets allowed': 43, 'Toys': 44, 'Hot tub': 45, 'Communal pool': 46, 'Video Games': 47, 'Fitness room/Equipment': 48, 'Heated pool': 49, 'Private pool': 50, 'Tennis': 51, 'Indoor pool

In [5]:

# Step 2 - scan and create the data
for index, row in cal.iterrows():
    #print(index)
    a = row['features']
    b = a.split('\n')
    
    save_features = [0] * len(position_dict)
    
    for b1 in b:    
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                save_features[b1_pos] = 1
            else:
                print("ERROR: found a new feature not in dictionary: '" + b1 + "'")
    #print("Index: " + index)
    #if ( index > 0 ):
    data_dict[index] = save_features
    
df1 = pd.DataFrame.from_dict(data_dict)


df2 = df1.T

col_n_1 = position_dict.keys()

col_n_2 = []


for c in col_n_1:
    # Cleaning text
    for char in '\'/-.,\n ':
        c=c.replace(char,'_')

    # All words lower case
    c = c.lower()
    
    col_n_2.append('f_' + c)

df2.columns = col_n_2

df2.head()

,f_books,f_child_s_high_chair,f_children_welcome,f_clothes_dryer,f_dining,f_dishes_and_utensils,f_dvd_player,f_games,f_hair_dryer,f_heating,...,f_pets_allowed,f_toys,f_hot_tub,f_communal_pool,f_video_games,f_fitness_room_equipment,f_heated_pool,f_private_pool,f_tennis,f_indoor_pool
4840533,0,0,0,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4841630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4845880,1,1,1,1,1,1,0,1,1,1,...,0,1,0,0,0,0,0,0,0,0
4848261,1,0,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4887073,0,0,0,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Write out the file to CSV so it can be joined in the main branch

csv_out = df2.to_csv()

fh = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_features/stayz_nsw_features_' + date_str + '.csv', 'w')

fh.write(csv_out)

fh.close()